In [10]:
import pandas as pd
import numpy as np

column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
df_iris = pd.read_csv("../HW4/resource/iris.data", header=None, names=column_names)

ptest = df_iris.iloc[149].copy()
df_iris.drop(149, inplace=True)
# df_iris.reset_index(drop=True, inplace=True)

print("ptest:")
print(ptest)

ptest:
sepal_length               5.9
sepal_width                3.0
petal_length               5.1
petal_width                1.8
species         Iris-virginica
Name: 149, dtype: object


In [11]:
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2) ** 2))


distances = []
for i, row in df_iris.iterrows():
    dist = euclidean_distance(ptest[['sepal_length', 'sepal_width',  'petal_length', 'petal_width']], row[['sepal_length', 'sepal_width',  'petal_length', 'petal_width']])
    distances.append((i, dist, row['species']))

distances_df = pd.DataFrame(distances, columns=['index', 'distance', 'species'])
# distances_df['index'].sort_values(ascending=False)
k = 3
nearest_neighbors = sorted(distances, key=lambda x: x[1])[:k]

for i, (idx, dist, species) in enumerate(nearest_neighbors):
    print(f"Index {idx}, Distance {dist:.4f}, Class {species}")



Index 127, Distance 0.2828, Class Iris-virginica
Index 138, Distance 0.3162, Class Iris-virginica
Index 101, Distance 0.3317, Class Iris-virginica


In [15]:
def z_score_normalize(df, test_point):
    df_normalized = df.copy()
    test_normalized = test_point.copy()

    for column in ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']:
        mean = df[column].mean()
        std = df[column].std()
        df_normalized[column] = (df[column] - mean) / std
        test_normalized[column] = (test_point[column] - mean) / std

    return df_normalized, test_normalized

df_iris_norm, ptest_norm = z_score_normalize(df_iris, ptest)

distances_norm = []
for i, row in df_iris_norm.iterrows():
    dist = euclidean_distance(ptest_norm[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']],
                             row[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
    distances_norm.append((i, dist, row['species']))

nearest_neighbors_norm = sorted(distances_norm, key=lambda x: x[1])[:k]

for i, (idx, dist, species) in enumerate(nearest_neighbors_norm):
    print(f"Index {idx}, Distance {dist:.4f}, Class {species}")


Index 138, Distance 0.2081, Class Iris-virginica
Index 127, Distance 0.2660, Class Iris-virginica
Index 70, Distance 0.4901, Class Iris-versicolor


In [14]:
from collections import Counter
predicted_class = Counter([neighbor[2] for neighbor in nearest_neighbors]).most_common(1)[0][0]
correct_without_norm = predicted_class == ptest['species']
predicted_class_norm = Counter([neighbor[2] for neighbor in nearest_neighbors_norm]).most_common(1)[0][0]


for i, (idx, dist, species) in enumerate(nearest_neighbors_norm):
    print(f"Index {idx}, Distance {dist:.4f}, Class {species}")

print(f"\nPrediction with normalization (k={k}):")
print(f"Neighbor classes: {[neighbor[2] for neighbor in nearest_neighbors_norm]}")
print(f"Predicted class: {predicted_class_norm}")
print(f"Actual class: {ptest['species']}")
print(f"Correct: {predicted_class_norm == ptest['species']}")

correct_with_norm = predicted_class_norm == ptest['species']

print("\n" + "="*50)
print("ACCURACY COMPARISON:")
print(f"Without normalization: {'CORRECT' if correct_without_norm else 'WRONG'}")
print(f"With normalization: {'CORRECT' if correct_with_norm else 'WRONG'}")

if correct_without_norm == correct_with_norm:
    if correct_without_norm:
        print("RESULT: Both methods produced CORRECT predictions")
    else:
        print("RESULT: Both methods produced WRONG predictions")
else:
    if correct_without_norm:
        print("RESULT: Normalization DECREASED accuracy")
    else:
        print("RESULT: Normalization INCREASED accuracy")

TypeError: 'bool' object is not iterable